In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import cartopy
import cartopy.crs as ccrs
import cftime
from datetime import date, timedelta

In [15]:
case = "fanv3_1th"
path = "/scratch/jinmuluo/archive/" + case + "/lnd/hist/" 
start_date = date(2010, 1, 1)
end_date = date(2013, 1, 1)
delta = end_date - start_date
fields = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL"]
fanCLM = []
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    day = day.strftime('%Y-%m-%d')
    fanCLM.append(path + case + ".clm2.h6." + day +"-00000.nc")
print(fanCLM[-1])

/scratch/jinmuluo/archive/fanv3_1th/lnd/hist/fanv3_1th.clm2.h6.2013-01-01-00000.nc


In [ ]:
def preprocess (ds, fields = fields):
    return(ds[fields])

def fix_time(ds):  
    ndays = len(ds.time)
    yr0 = ds['time.year'][0].values
    ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
dsCLM

In [ ]:
def sp_plot(ds, field, coords, show=False):
    sp = ds.sel(lat=coords[0], lon=coords[1], method='nearest')    
    # gN/m2/s to kg/day
    result = sp[field] * ds['landfrac'] * ds['area'] * 1000 * 24 * 3600
    result.plot(hue='sim', x='time')
    if show:
        result.plot(x='time')
        plt.show()
    return result

In [ ]:
def tot_plot(ds, field, show=False):
    # calculate global total emission (kg/day)
    total = ds[field]*ds['area']* 1000 * 24 * 3600
    total.sum(skipnan=True, dim=[1, 2])
    if show:
        total.plot(x='time')
        plt.show()
    return total

In [ ]:
def regional_plot(ds, field, coords, show=False):
    x = slice(coords['lower_lat'], coords['upper_lat'])
    y = slice(coords['west_lon'], coords['east_lon'])
    sp = ds[field].sel(lat=x, lon=y).sum(skipnan=True, dim=[1, 2])
    if show:
        sp.plot(x='time')
        plt.show()
    return 0

In [ ]:
coordinates = [23.45, 130.25]
sp_plot(ds=dsCLM, field=fields[0], coords=coordinates, show=True)

In [ ]:
tot_plot(ds=dsCLM, field=fields[0], show=True)

In [3]:
coordinates = {'upper_lat':45.50, 'lower_lat':25.35, 'west_lon': 100.00, 'east_lon':130.00}
regional_plot(ds=dsCLM, field=fields[0], coords=coordinates, show=True)

NameError: name 'regional_plot' is not defined